In [4]:
from pandas import Timestamp
from pyspark.sql.functions import to_timestamp,from_unixtime,from_utc_timestamp,col
from ts.flint import FlintContext
from pandas import date_range,read_csv,DataFrame
from pyspark.sql.functions import lit
flintContext = FlintContext(sqlContext)
#df1 = tarfea.toPandas()
#df2 = df1.resample(freq).sum()/1
#df = tarfea.fillna(0).toPandas().set_index('time',inplace=True)
period = 'quick'
source ='sdg'
if (period=='full'):
    start_training = Timestamp("2019-02-01 00:00:00", freq='H')
    end_training = Timestamp("2019-02-15 00:00:00", freq='H')
    start_testing = Timestamp("2019-02-15 00:00:00", freq='H')
    end_testing = Timestamp("2019-02-28 00:00:00", freq='H')
    period_sdg_partition='ymd=201902*'
    period_qa_partition='year=2019/month=2/day=*'
    d =spark.createDataFrame(DataFrame(date_range('2019-02-01', periods=24*28, freq='H')),['time'])
else:
    start_training = Timestamp("2019-02-01 00:00:00", freq='H')
    end_training = Timestamp("2019-02-02 00:00:00", freq='H')
    start_testing = Timestamp("2019-02-02 00:00:00", freq='H')
    end_testing = Timestamp("2019-02-03 00:00:00", freq='H')
    period_sdg_partition='ymd=20190201'
    period_qa_partition='year=2019/month=2/day=1'
    d =spark.createDataFrame(DataFrame(date_range('2019-02-01', periods=24, freq='H')),['time'])
    #sans = ['DSS36791010','DSS32702231','DSS35502688','DSS35473329']
statecodes =['calls','24.1.1','11.3.1','14.1.1','12.7.1','12.7.2']
target_files="s3://sfdmdb/SF_CASE_VW/"+period_sdg_partition+"/0.avro"
feature_parquet_files="s3://qaessential/jup*_stats.vsat_stcd/"+period_sdg_partition+"/*"
if (source=='sdg'):
    feature_avro_files="s3://qaessential/jup*_stats.vsat_stcd/"+period_sdg_partition+"/*"
else:
    feature_avro_files="s3://qaessential/jup*_stats_vsat_stcd/"+period_qa_partition+"/*"

In [7]:
t = spark.read.format("avro").load(target_files).select(from_utc_timestamp("createddate","UTC").alias('time'),col('account_san__c').alias('san'),'case_subtype__c').filter(col("case_subtype__c").isin("Retention","Performance","PS TAD"))
#target_count = target.groupBy("case_subtype__c").count().show()
ta = flintContext.read.dataframe(t)
#tar = ta.filter(col("san").isin(sans)).show()
#f = spark.read.format('avro').load(feature_avro_files).select(to_timestamp(from_unixtime("collection_start_time", "yyyy-MM-dd HH")).alias('time'),col('device_id').alias('san'),'statecode','statecodeduration')
f = spark.read.parquet(feature_parquet_files).select(to_timestamp(from_unixtime("collection_start_time", "yyyy-MM-dd HH")).alias('time'),col('device_id').alias('san'),'statecode','statecodeduration')
fe= flintContext.read.dataframe(f)
#fea = fe.filter(col("san").isin(sans)).show()
ta.cache()
fe.cache()

'JavaMember' object has no attribute 'sessionState'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 383, in __repr__
    if not self._support_repr_html and self.sql_ctx._conf.isReplEagerEvalEnabled():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/context.py", line 99, in _conf
    return self.sparkSession._jsparkSession.sessionState().conf()
AttributeError: 'JavaMember' object has no attribute 'sessionState'



In [ ]:
from pyspark.sql.functions import least
s = ta.select('san')
ds = d.crossJoin(s).sort('time')
h=flintContext.read.dataframe(ds)
#tar=ta.filter(col("san").isin(sans)).withColumn('statecode',lit('calls')).withColumn('statecodeduration',lit(1)).drop('case_subtype__c')
#fea=fe.filter(col("san").isin(sans) & col("statecode").isin(statecodes))
tar=ta.withColumn('statecode',lit('calls')).withColumn('statecodeduration',lit(1)).drop('case_subtype__c')
fea=fe.filter(col("statecode").isin(statecodes)).withColumn('statecodeduration',least(lit(1),col('statecodeduration')/3600))

In [ ]:
from pyspark.sql.functions import date_trunc
from pyspark.ml.feature import MinMaxScaler
tf=tar.union(fea).withColumn('time',date_trunc('hour','time'))
tf=tf.groupBy('san','time').pivot('statecode').max('statecodeduration').sort('san','time')
#tarfea=h.leftJoin(tf,key='san',tolerance='59m').sort('san','time')
#piv_fea=tarfea.groupBy('san','time').pivot('statecode').max('statecodeduration').sort('san','time')
#piv_fea=h.join(tf, (h.san==tf.san) & (h.time==tf.time),how='left')
piv_fea=h.join(tf, ['san','time'],how='left')
df = piv_fea.fillna(0).toPandas().set_index(['san','time'])
df.sort_values(['san','time'],inplace=True)
#df.columns=df.columns.str.replace('_','.')
df.describe()

In [ ]:
sans = s.rdd.flatMap(lambda x: x).collect()
training_data = [
    {
        "start": str(start_training),
        "target": df.loc[san].loc[start_training:end_training - 1]['calls'].tolist(),  # We use -1, because pandas indexing includes the upper bound 
        "cat": san,
        "dynamic_feat": [df.loc[san].loc[start_training:end_training - 1]['11.3.1'].tolist(),df.loc[san].loc[start_training:end_training - 1]['14.1.1'].tolist()] # We use -1, because pandas indexing includes the upper bound 
    }
    for san in sans 
]
testing_data = [
    {
        "start": str(start_testing),
        "target": df.loc[san].loc[start_testing:end_testing - 1]['calls'].tolist(),  # We use -1, because pandas indexing includes the upper bound
        "dynamic_feat": [df.loc[san].loc[start_testing:end_testing - 1]['11.3.1'].tolist(),df.loc[san].loc[start_testing:end_testing - 1]['14.1.1'].tolist()] # We use -1, because pandas indexing includes the upper bound 
    }
    for san in sans 
]
print(len(training_data))
print(len(testing_data))
import json
import boto3
import s3fs
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))
write_dicts_to_file('/var/lib/zeppelin/train.json',training_data)
write_dicts_to_file('/var/lib/zeppelin/test.json',testing_data)
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{} already exists.\nSet override to upload anyway.\n'.format(s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)
s3_data_path='s3://jupstats'
copy_to_s3("/var/lib/zeppelin/train.json", s3_data_path + "/train/train.json",True)
copy_to_s3("/var/lib/zeppelin/test.json", s3_data_path + "/test/test.json",True)
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")
with s3filesystem.open(s3_data_path + "/test/test.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")